## 匿名函数
当我们在传入函数时，有些时候，不需要显式地定义函数，直接传入匿名函数更方便。

在Python中，对匿名函数提供了有限支持。还是以map()函数为例，计算$f(x)=x^{2}$时，除了定义一个f(x)的函数外，还可以直接传入匿名函数：

In [1]:
list(map(lambda x: x * x, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

    匿名函数lambda x: x * x实际上就是：  
    def f(x):  
       return x * x

关键字lambda表示匿名函数，冒号前面的x表示函数参数。

匿名函数有个限制，就是只能有一个表达式，不用写return，返回值就是该表达式的结果。

用匿名函数有个好处，因为函数没有名字，不必担心函数名冲突。此外，匿名函数也是一个函数对象，也可以把匿名函数赋值给一个变量，再利用变量来调用该函数：

In [2]:
f = lambda x : x * x 
f(5)

25

同样，也可以把匿名函数作为返回值返回，比如：

In [3]:
def build(x, y):
    return lambda: x * x + y * y
z = build(3,5)
z()


34

## 装饰器
由于函数也是一个对象，而且函数对象可以被赋值给变量，所以，通过变量也能调用该函数

In [4]:
def time():
    print('time time time ')
f = time
f()

time time time 


函数对象有一个__name__属性，可以拿到函数的名字：

In [5]:
print(time.__name__)
print(f.__name__ )

time
time


现在，假设我们要增强time()函数的功能，但又不希望修改time()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）
本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：

In [6]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)  #可变参数 关键字参数
    return wrapper

观察上面的log，因为它是一个decorator，所以接受一个函数作为参数，并返回一个函数。我们要借助Python的@语法，把decorator置于函数的定义处：

In [7]:
@log #把@log放到ptime()函数的定义处，相当于执行了语句：ptime = log(ptime)
def ptime():
    print('time time time')
ptime()

call ptime():
time time time


由于log()是一个decorator，返回一个函数，所以，原来的ptime()函数仍然存在，只是现在同名的ptime变量指向了新的函数，于是调用ptime()将执行新函数，即在log()函数中返回的wrapper()函数。

wrapper()函数的参数定义是(\*args, \**kw)，因此，wrapper()函数可以接受任意参数的调用。在wrapper()函数内，首先打印日志，再紧接着调用原始函数。

如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。比如，要自定义log的文本：

In [8]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

@log('execute')
def ptime():
    print('ttttt')
print(ptime.__name__)

wrapper


因为返回的那个wrapper()函数名字就是'wrapper'，所以，需要把原始函数的__name__等属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错。

不需要编写wrapper.__name__ = func.__name__这样的代码，Python内置的functools.wraps就是干这个事的，所以，一个完整的decorator的写法如下：

In [9]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

### 偏函数
Python的functools模块提供了很多有用的功能，其中一个就是偏函数（Partial function）
functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单

In [10]:
import functools
int2 = functools.partial(int ,base=2 )
int2('100001')

33